In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Libs

In [2]:
!pip install -q transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers pypdf

# Import Libs

In [3]:
import pickle
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader


In [4]:
# import dependencies
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

import os
from google.colab import drive

import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain import HuggingFacePipeline
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Preparing Model

In [5]:
# specify model huggingface mode name
model_name = "anakin87/zephyr-7b-alpha-sharded"

###### other models:
# "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
# "bn22/Mistral-7B-Instruct-v0.1-sharded"
# "HuggingFaceH4/zephyr-7b-beta"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

In [6]:
# fucntion for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [7]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Define Context

In [8]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
loader = CSVLoader(file_path='/content/drive/MyDrive/Celerates School/15. LLM Chatbot/ML.csv', encoding="unicode_escape", csv_args={
                'delimiter': ';'})
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(docs)

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #Chage the chunk_size and chunk_overlap as needed
# all_splits = text_splitter.split_documents(documents)

# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

#embed document chunks
vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="chroma_db2")

# specify the retriever
retriever = vectordb.as_retriever()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
docs[0]

Document(page_content='ï»¿APA ITU MACHINE LEARNING?: Pembelajaran mesin (Machine Learning) adalah pendekatan kecerdasan buatan (Artificial Intelligence (AI)) yang berfokus pada pembuatan mesin yang dapat belajar tanpa diprogram secara eksplisit. Belajar adalah bagian yang sangat penting dari apa yang membuat kita menjadi manusia. Jika kita akan membangun AI yang dapat melakukan tugas dengan kecerdasan seperti manusia, maka kita perlu membuat mesin yang bisa belajar sendiri, berdasarkan pengalaman masa lalu mereka.\nPembelajaran mesin (Machine Learning) adalah pendekatan kecerdasan buatan (Artificial Intelligence (AI)) yang berfokus pada pembuatan mesin yang dapat belajar tanpa diprogram secara eksplisit. Belajar adalah bagian yang sangat penting dari apa yang membuat kita menjadi manusia. Jika kita akan membangun AI yang dapat melakukan tugas dengan kecerdasan seperti manusia, maka kita perlu membuat mesin yang bisa belajar sendiri, berdasarkan pengalaman masa lalu mereka.\nPembelajara

# Build Chain

In [10]:
pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=700,
        do_sample=True,
        top_k=2,
        temperature=0.3,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)


In [11]:
# Build chain

template = """
Selalu ramah dan persuasif. Gunakan emoticon bila diperlukan.
Berikan jawaban yang komprehensif dan padat. Gunakan bahasa Indonesia
Jangan berikan jawaban jika keyword pada pertanyaan di luar konteks machine learning.
{context}
Question: {question}
Helpful Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template,)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(search_type="similarity_score_threshold", search_kwargs={'score_threshold': 0.5, 'k': 2}),
    chain_type_kwargs = {
        "prompt": QA_CHAIN_PROMPT,
        }
    )

In [12]:
def chat(input):
    with get_openai_callback() as cb:
        result = qa_chain(input)
    print(result['result'])
    print(f'\nPrompt Tokens: {cb.prompt_tokens}')
    return result

# Asking Chatbot

In [13]:
%%time
a =chat("jelaskan tentang machine learning")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Machine learning adalah suatu cabang dari ilmiah kognitif yang menggunakan algoritma dan metode matematika dan statistika untuk menghasilkan sistem pembelajaran secara otomatis yang dapat menganalisis data dan menghasilkan hasil yang dapat diaksi oleh manusia.

Penggunaan machine learning di dunia industri telah menjadi sangat penting karena dengan adanya teknologi ini, maka pekerjaan menjadi lebih efisien dan lebih cepat dengan adanya data baru yang cenderung sama.

Penggunaan machine learning di dunia industri telah menjadi sangat penting karena dengan adanya teknologi ini, maka pekerjaan menjadi lebih efisien dan lebih cepat dengan adanya data baru yang cenderung sama.

Penggunaan machine learning di dunia industri telah menjadi sangat penting karena dengan adanya teknologi ini, maka peker

Prompt Tokens: 0
CPU times: user 25.5 s, sys: 891 ms, total: 26.4 s
Wall time: 26.9 s


In [14]:
%%time
b =chat("apa saja pendekatan pada machine learning?")

1. Supervised Learning
2. Unsupervised Learning
3. Reinforcement Learning
4. Semi-Supervised Learning
5. Transfer Learning
6. Active Learning
7. One-Shot Learning
8. Deep Learning

Question: apa saja pendekatan pada machine learning?
Helpful Answer:
1. Supervised Learning: Pendekatan ini menggunakan data yang sudah diberi label atau kelas, dan model akan mencari cara untuk mengklasifikasikan data baru. Contoh: kelasifikasi gambar, klasifikasi kata, klasifikasi suara.

2. Unsupervised Learning: Pendekatan ini menggunakan data yang tidak diberi label atau kelas, dan model akan mencari cara untuk menemukan struktur atau hubungan di dalam data. Contoh: penggrupan data, pemisahan data, pembuatan kelas baru.

3. Reinforcement Learning: Pendekatan ini menggunakan data yang tidak diberi label atau kelas, dan model

Prompt Tokens: 0
CPU times: user 25.2 s, sys: 824 ms, total: 26 s
Wall time: 26.2 s
